In [11]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from matplotlib import pyplot as plt
from sklearn.model_selection import TimeSeriesSplit



# Assuming your data is in a pandas dataframe called 'df'
# with 'date' as the date column and 'quantity' as the quantity column
df = pd.read_csv('new_df.csv')

# Convert the 'date' column to datetime format if it's not already
df['everyday'] = pd.to_datetime(df['everyday'])

# Set the date column as the index
df.set_index('everyday', inplace=True)


# Split the data into training and testing sets (optional)
# You can experiment with different split ratios
train_size = int(len(df) * 0.8)
train, test = df[:train_size], df[train_size:]

# Define hyperparameter search space (optional)
# Experiment with different values for these parameters
param_grid = {
    'seasonal_periods': range(2, 13),  # Try different seasonality periods
    'trend': ['additive', 'multiplicative']  # Try both additive and multiplicative trends
}

# Time Series Split for cross-validation (handles time order)
tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits for desired folds

# Manual hyperparameter tuning loop (optional)
# Iterate through param_grid and fit models for each parameter combination

best_mse = float('inf')  # Initialize to high value
for params in param_grid.values():
    model = ExponentialSmoothing(train['quantity'], **params)
    for train_index, val_index in tscv.split(train):
        model_fit = model.fit(train.iloc[train_index])
        mse = ((train.iloc[val_index] - model_fit.predict(start=val_index[0], end=val_index[-1]))**2).mean()
        if mse < best_mse:
            best_mse = mse
            best_model = model_fit
            best_params = params

# Generate forecasts for 3 months (90 days)
future_dates = train.index[-1] + pd.DateOffset(days=90)
forecast = best_model.predict(start=train.index[-1], end=future_dates)

# Calculate Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) on the test set (optional)
mse = ((test['quantity'] - best_model.predict(start=test.index[0], end=test.index[-1]))**2).mean()
rmse = np.sqrt(mse)  # Calculate RMSE from MSE
print('Mean Squared Error:', round(mse, 2))
print('Root Mean Squared Error:', round(rmse, 2))

# Visualize actual vs predicted values
plt.figure(figsize=(12, 6))  # Adjust figure size for better visualization
train['quantity'].plot(label='Actual')
forecast.plot(label='Forecast', alpha=0.7)
plt.legend()
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.title('Actual vs. Forecasted Quantity')
plt.grid(True)
plt.show()




TypeError: statsmodels.tsa.holtwinters.model.ExponentialSmoothing() argument after ** must be a mapping, not range

In [8]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.
